In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys  
sys.path.insert(0, '..')

In [34]:
from utils import get_tafeng_graph, load_weights
from knowledge_graph.datasets import KgPosNegTriples, TimeSplittedDataset, KgCustomers, EdgeWithType
from knowledge_graph.layer_generators import LayerNodeGenerator
from utils import get_dates_for_split, get_graph_splits, get_test_interactions, load_tafeng_dataframe
from models.Model import Model
from models.config import Config
import numpy as np
import random
from tqdm.notebook import tqdm
import sys
import torch
import torch.optim as optim
from train import train_transR_one_epoch, train_lstm_one_epoch, evaluate
from datetime import datetime

In [4]:
knowledge_graph = get_tafeng_graph(user_k_core=2, item_k_core=1)

2021-05-12 13:26:16,156 - TaFengGraph - [INFO] - loading entities
2021-05-12 13:26:16,427 - TaFengGraph - [INFO] - loading relations
2021-05-12 13:27:55,541 - TaFengGraph - [INFO] - loaded purchase
2021-05-12 13:28:58,236 - TaFengGraph - [INFO] - loaded bought_in
2021-05-12 13:30:00,504 - TaFengGraph - [INFO] - loaded belongs_to_age_group
2021-05-12 13:31:04,691 - TaFengGraph - [INFO] - loaded belongs_to_subclass


In [5]:
timestamps = knowledge_graph.relation_set.get_all_timestamps()
splitting_points = get_dates_for_split(timestamps, n_points=5)

In [6]:
splits = get_graph_splits(knowledge_graph, splitting_points)
del timestamps, splitting_points

2021-05-12 13:31:06,233 - TaFengGraph - [INFO] - converting belongs_to_subclass
2021-05-12 13:31:06,713 - TaFengGraph - [INFO] - converting bought_in
2021-05-12 13:31:07,952 - TaFengGraph - [INFO] - converting purchase
2021-05-12 13:31:09,570 - TaFengGraph - [INFO] - converting belongs_to_age_group
2021-05-12 13:31:10,012 - TaFengGraph - [INFO] - converting belongs_to_subclass
2021-05-12 13:31:10,198 - TaFengGraph - [INFO] - converting bought_in
2021-05-12 13:31:11,117 - TaFengGraph - [INFO] - converting purchase
2021-05-12 13:31:12,832 - TaFengGraph - [INFO] - converting belongs_to_age_group
2021-05-12 13:31:13,062 - TaFengGraph - [INFO] - converting belongs_to_subclass
2021-05-12 13:31:13,602 - TaFengGraph - [INFO] - converting bought_in
2021-05-12 13:31:14,250 - TaFengGraph - [INFO] - converting purchase
2021-05-12 13:31:16,009 - TaFengGraph - [INFO] - converting belongs_to_age_group
2021-05-12 13:31:16,241 - TaFengGraph - [INFO] - converting belongs_to_subclass
2021-05-12 13:31:16,

In [7]:
train_splits = splits[:-1]
test_split = splits[-1]

In [8]:
pos_neg_triples_ds = [KgPosNegTriples(split) for split in train_splits]
ts_ds = TimeSplittedDataset(pos_neg_triples_ds)

In [9]:
customer_indices = list(map(lambda x: knowledge_graph.entity_set.entity2idx[x], knowledge_graph.entity_set.customer))
product_indices = list(map(lambda x: knowledge_graph.entity_set.entity2idx[x], knowledge_graph.entity_set.product))

In [23]:
def test_pos_neg_triples(split):
    batch_size = 100
    test_kg_pos_neg_triples = KgPosNegTriples(split)
    n_batches = len(test_kg_pos_neg_triples) // batch_size + 1
    batch_idx = 0
    for _ in tqdm(range(n_batches)):
        batch = test_kg_pos_neg_triples[batch_idx: batch_idx + batch_size]
        for idx in range(batch_size):
            assert EdgeWithType(relation_type=batch.relation[idx], to=batch.neg_tail[idx]) not in split[batch.head[idx]]
            assert EdgeWithType(relation_type=batch.relation[idx], to=batch.pos_tail[idx]) in split[batch.head[idx]]

In [24]:
test_pos_neg_triples(test_split)

In [32]:
def test_customer_ds(splits):
    purchase_relation_idx=knowledge_graph.relation_set.relation2idx['purchase']
    customer_ds = KgCustomers(
        splits=train_splits,
        customer_indices=customer_indices,
        product_indices=product_indices,
        purchase_relation_idx=purchase_relation_idx
    )
    
    batch_size = 100
    n_batches = len(customer_ds) // batch_size + 1
    batch_idx = 0
    for _ in tqdm(range(n_batches)):
        batch = customer_ds[batch_idx: batch_idx + batch_size]
        pos_products = batch.pos_products.numpy()
        neg_products = batch.neg_products.numpy()
        customers = batch.customer
        for idx in range(batch_size):
            for time_idx in range(len(splits)-1):
                to_p = pos_products[idx][time_idx]
                to_n = neg_products[idx][time_idx]
                assert EdgeWithType(relation_type=purchase_relation_idx, to=to_p) in splits[time_idx+1][customers[idx]]
                assert EdgeWithType(relation_type=purchase_relation_idx, to=to_n) not in splits[time_idx+1][customers[idx]]    

In [33]:
test_customer_ds(train_splits)

In [35]:
tafeng_df = load_tafeng_dataframe(2, 1)

In [41]:
tafeng_df

,date,customer,age_group,zip_code,product_subclass,product
0,2000-11-01,1104905,45-49,115,110411,4710199010372
1,2000-11-01,418683,45-49,115,120107,4710857472535
2,2000-11-01,1057331,35-39,115,100407,4710043654103
3,2000-11-01,1849332,45-49,Others,120108,4710126092129
4,2000-11-01,1981995,50-54,115,100205,4710176021445
...,...,...,...,...,...,...
793834,2001-02-28,312790,35-39,114,530501,4713317035042
793835,2001-02-28,57486,40-44,115,530209,4710731060124
793836,2001-02-28,733526,>65,Unknown,510539,4716340052307
793837,2001-02-28,173704,45-49,115,520457,4714276145315


In [46]:
for idx, row in tafeng_df.iterrows():
    print(row.values)
    break

[Timestamp('2000-11-01 00:00:00') '1104905' '45-49' '115' '110411'
 '4710199010372']


In [51]:
def test_graph_structure():
    adj_lists = knowledge_graph.to_adj_lists()
    
    def assert_both_edges_exist(v1: int, v2: int, relation_type: str):
        assert EdgeWithType(to=v2, relation_type=relation_type) in adj_lists[v1]
        assert EdgeWithType(to=v1, relation_type=relation_type) in adj_lists[v2]
    
    
    tafeng_df = load_tafeng_dataframe(user_k_core=2, item_k_core=1)[:1000]

    entity2idx = knowledge_graph.entity_set.entity2idx
    relation2idx = knowledge_graph.relation_set.relation2idx
    for _, row in tqdm(tafeng_df.iterrows()):
        customer, product, age_group, zip_code, product_subclass = (
            entity2idx[str(row.customer)], entity2idx[str(row['product'])], entity2idx[str(row.age_group)],
            entity2idx[str(row.zip_code)], entity2idx[str(row.product_subclass)]
        )
        
        assert_both_edges_exist(product, customer, relation2idx['purchase'])
        assert_both_edges_exist(age_group, customer, relation2idx['belongs_to_age_group'])
        assert_both_edges_exist(zip_code, product, relation2idx['bought_in'])
        assert_both_edges_exist(product, product_subclass, relation2idx['belongs_to_subclass'])

In [52]:
test_graph_structure()

2021-05-12 15:24:43,629 - TaFengGraph - [INFO] - converting belongs_to_subclass
2021-05-12 15:24:43,821 - TaFengGraph - [INFO] - converting bought_in
2021-05-12 15:24:44,470 - TaFengGraph - [INFO] - converting purchase
2021-05-12 15:24:53,806 - TaFengGraph - [INFO] - converting belongs_to_age_group
